In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
        y = torch.FloatTensor([int(self.y[index])])
        return x, y

    def __len__(self):
        return self.length

In [3]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
          nn.Linear(3, 1),
          nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [4]:
dataset = CustomDataset("../datasets/binary.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size], torch.manual_seed(4))
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [6]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 0.0028, -0.0006,  0.0036]], requires_grad=True), Parameter containing:
tensor([0.0949], requires_grad=True)], Cost : 0.680
Epoch : 2000, Model : [Parameter containing:
tensor([[0.0034, 0.0005, 0.0041]], requires_grad=True), Parameter containing:
tensor([-0.0346], requires_grad=True)], Cost : 0.666
Epoch : 3000, Model : [Parameter containing:
tensor([[0.0045, 0.0017, 0.0051]], requires_grad=True), Parameter containing:
tensor([-0.1609], requires_grad=True)], Cost : 0.652
Epoch : 4000, Model : [Parameter containing:
tensor([[0.0052, 0.0017, 0.0053]], requires_grad=True), Parameter containing:
tensor([-0.2843], requires_grad=True)], Cost : 0.636
Epoch : 5000, Model : [Parameter containing:
tensor([[0.0059, 0.0036, 0.0064]], requires_grad=True), Parameter containing:
tensor([-0.4046], requires_grad=True)], Cost : 0.627
Epoch : 6000, Model : [Parameter containing:
tensor([[0.0065, 0.0034, 0.0068]], requires_grad=True), Parameter contain

In [10]:
with torch.no_grad():
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)
        
        outputs = model(x)

        print("x:", x)
        print("y:", y)
              
        print(outputs)
        print(outputs >= torch.FloatTensor([0.5]).to(device))
        print("--------------------")

x: tensor([[54., 69., 52.],
        [78., 87., 26.],
        [81., 67., 94.],
        [83., 60., 59.]])
y: tensor([[0.],
        [0.],
        [1.],
        [1.]])
tensor([[0.6221],
        [0.6426],
        [0.7555],
        [0.6813]])
tensor([[True],
        [True],
        [True],
        [True]])
--------------------
x: tensor([[ 61.,   9.,  16.],
        [ 25.,  80.,  75.],
        [ 42.,  23.,   2.],
        [ 75., 100.,  76.]])
y: tensor([[0.],
        [0.],
        [0.],
        [1.]])
tensor([[0.4486],
        [0.6318],
        [0.3985],
        [0.7563]])
tensor([[False],
        [ True],
        [False],
        [ True]])
--------------------
x: tensor([[16., 35., 65.],
        [99., 63., 97.],
        [54., 74., 68.],
        [24., 91.,  0.]])
y: tensor([[0.],
        [1.],
        [1.],
        [0.]])
tensor([[0.5122],
        [0.7840],
        [0.6654],
        [0.4713]])
tensor([[ True],
        [ True],
        [ True],
        [False]])
--------------------
x: tensor([

In [30]:
with torch.no_grad():
    model.eval()
    
    x = torch.tensor([[85., 92., 75.],
        [75., 64., 50.],
        [38., 58., 63.],
        [23., 15., 32.]]).to(device)
    
    y = torch.tensor([[1.], [1.], [0.], [0.]]).to(device)
    

    print("x:", x)
    print("y:", y)
          
    outputs = model(x)
    print(outputs)
    print(outputs >= torch.FloatTensor([0.5]).to(device))
    print("--------------------")

x: tensor([[85., 92., 75.],
        [75., 64., 50.],
        [38., 58., 63.],
        [23., 15., 32.]])
y: tensor([[1.],
        [1.],
        [0.],
        [0.]])
tensor([[0.7605],
        [0.6525],
        [0.5952],
        [0.4141]])
tensor([[ True],
        [ True],
        [ True],
        [False]])
--------------------
